In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dateutil

In [33]:
animals_used = [1091, 1093, 1060, 1062, 1074, 1092, 1102, 1076, 1082, 1101]
diabetic = [1076, 1082, 1101]
impaired = [1060, 1062, 1074, 1092, 1102]
normal = [1091, 1093]
bg_dict = {'diabetic': diabetic, 'impaired': impaired, 'normal': normal}
animal_to_tolerance_dict = {1076: 'diabetic', 1082: 'diabetic',1101: 'diabetic',
                            1060: 'impaired', 1062: 'impaired', 1074: 'impaired', 1092: 'impaired', 1102: 'impaired', 
                            1091: 'normal', 1093: 'normal',}

obs = pd.read_excel(r'..\data\metadata\20210421_FBGRBGOGTT_Huishi sample sheet.xlsx', sheet_name='cleaned')
obs['animal'] = obs['animal'].str.strip('TM').astype('int')
obs['datetime'] = obs['when'].apply(dateutil.parser.parse)
animal_phenotypes = pd.read_excel(r'..\data\metadata\20210421_FBGRBGOGTT_Huishi sample sheet.xlsx', sheet_name='animal phenotypes')
animal_phenotypes['animal'] = animal_phenotypes['animal'].str.strip('TM').astype('int')

metab_file_grouping = pd.read_csv(r'..\data\metadata\20210831_KAO_HILIC_Huishi_sequence.csv',
                               skiprows=1)
metab_file_grouping['analytical_order'] = list(range(0,69))
metab_file_grouping = metab_file_grouping[['File Name', 'analytical_order']]

bad_file = '20210831_KAO_HILIC_ACN'  # file size is only 800 kB and it failed to convert in MSConvert
typo_files = {'20210831_KAO_HILIC_T193M20210406_9wk_RBG': '20210831_KAO_HILIC_T1093M_20210406_9wk_RBG',
              '20210831_KAO_HILIC_T1092_20210410_10wk_FBG': '20210831_KAO_HILIC_T1092M_20210410_10wk_FBG',
              '20210831_KAO_HILIC_T112M_20210330_8wk_RBG': '20210831_KAO_HILIC_T1102M_20210330_8wk_RBG',
              '20210831_KAO_HILIC_T1092M_20210402_wk_FBG': '20210831_KAO_HILIC_T1092M_20210402_9wk_FBG'}
qc_files = list(metab_file_grouping[metab_file_grouping['File Name'].str.contains('human')]['File Name'].values)
blank_files = ['20210831_KAO_HILIC_ACN', '20210831_KAO_HILIC_ACN_20210831232610',
               '20210831_KAO_HILIC_extraction_blank_1', '20210831_KAO_HILIC_extraction_blank_1_20210901004002']

def fix_filename(original_filename):
    if original_filename in typo_files:
        return typo_files[original_filename]
    else:
        return original_filename

metab_file_grouping['fixed_filename'] = metab_file_grouping['File Name'].apply(fix_filename)

metab_file_grouping['qc_file'] = metab_file_grouping['File Name'].isin(qc_files)
metab_file_grouping['quant_file'] = metab_file_grouping['fixed_filename'].str.contains('HILIC_T')
metab_file_grouping['blank_file'] = metab_file_grouping['File Name'].isin(blank_files)
metab_file_grouping = metab_file_grouping.drop(0)  # File is 800 kB and doesn't open

metab_file_grouping['animal'] = metab_file_grouping['fixed_filename'].str.split('HILIC_T').str[1].str.split('M_2021').str[0]
metab_file_grouping['week'] = metab_file_grouping[metab_file_grouping['quant_file'] == True]\
                                 ['fixed_filename'].str.split('wk_').str[0].str.split('_').str[-1]
metab_file_grouping['bg_type'] = metab_file_grouping[metab_file_grouping['quant_file'] == True]\
                                   ['fixed_filename'].str.split('_').str[-1]
metab_file_grouping['glucose_tolerance'] = metab_file_grouping['animal'].astype('float').astype('Int64').map(animal_to_tolerance_dict)

metab_file_grouping.to_csv(r'..\data\metadata\metabolomics_file_grouping.csv', index=False)

In [34]:
metab_file_grouping = pd.read_csv(r'..\data\metadata\metabolomics_file_grouping.csv')
metab_file_grouping

,File Name,analytical_order,fixed_filename,qc_file,quant_file,blank_file,animal,week,bg_type,glucose_tolerance
0,20210831_KAO_HILIC_ACN_20210831232610,1,20210831_KAO_HILIC_ACN_20210831232610,False,False,True,NaN,NaN,NaN,NaN
1,20210831_KAO_HILIC_extraction_blank_1,2,20210831_KAO_HILIC_extraction_blank_1,False,False,True,NaN,NaN,NaN,NaN
2,20210831_KAO_HILIC_extraction_blank_1_20210901...,3,20210831_KAO_HILIC_extraction_blank_1_20210901...,False,False,True,NaN,NaN,NaN,NaN
3,20210831_KAO_HILIC_human_plasma_1,4,20210831_KAO_HILIC_human_plasma_1,True,False,False,NaN,NaN,NaN,NaN
4,20210831_KAO_HILIC_T1060M_20210401_9wk_RBG,5,20210831_KAO_HILIC_T1060M_20210401_9wk_RBG,False,True,False,1060.0,9.0,RBG,impaired
...,...,...,...,...,...,...,...,...,...,...
63,20210831_KAO_HILIC_T1060M_20210408_10wk_RBG,64,20210831_KAO_HILIC_T1060M_20210408_10wk_RBG,False,True,False,1060.0,10.0,RBG,impaired
64,20210831_KAO_HILIC_T1102M_20210327_8wk_FBG,65,20210831_KAO_HILIC_T1102M_20210327_8wk_FBG,False,True,False,1102.0,8.0,FBG,impaired
65,20210831_KAO_HILIC_T1074M_20210322_8wk_FBG,66,20210831_KAO_HILIC_T1074M_20210322_8wk_FBG,False,True,False,1074.0,8.0,FBG,impaired
66,20210831_KAO_HILIC_T1082M_20210401_9wk_RBG,67,20210831_KAO_HILIC_T1082M_20210401_9wk_RBG,False,True,False,1082.0,9.0,RBG,diabetic


In [41]:
metabolomics_seq = pd.read_csv(r'..\data\metadata\20210831_KAO_HILIC_Huishi_sequence.csv',
                               skiprows=1)

pos = pd.read_csv(r'..\data\search\msdial\pos_alignment\Area_1_2021931826.txt', 
                     skiprows=4, sep='\t', index_col=0)
pos

,Average Rt(min),Average Mz,Metabolite name,Adduct type,Post curation result,Fill %,MS/MS assigned,Reference RT,Reference m/z,Formula,...,20210831_KAO_HILIC_T1101M_20210413_10wk_RBG,20210831_KAO_HILIC_T1102M_20210327_8wk_FBG,20210831_KAO_HILIC_T1102M_20210402_9wk_FBG,20210831_KAO_HILIC_T1102M_20210406_9wk_RBG,20210831_KAO_HILIC_T1102M_20210410_10wk_FBG,20210831_KAO_HILIC_T1102M_20210413_10wk_RBG,20210831_KAO_HILIC_T112M_20210330_8wk_RBG,20210831_KAO_HILIC_T193M20210406_9wk_RBG,1,1.1
Alignment ID,,,,,,,,,,,,,,,,,,,,,
0,17.980,70.06570,Unknown,[M+H]+,NaN,0.015,False,NaN,NaN,NaN,...,45086,66506,40004,95623,36121,41105,233175,335375,1.460943e+05,97831.170925
1,13.068,70.06570,Unknown,[M+H]+,found in higher mz's MsMs_950; found in higher...,0.735,False,NaN,NaN,NaN,...,377486,275506,240079,355702,265254,396101,340758,449559,3.151393e+05,107334.863624
2,17.361,70.06573,Unknown,[M+H]+,NaN,0.103,False,NaN,NaN,NaN,...,738587,401786,307830,215860,440384,600481,672270,620394,4.846335e+05,221611.902910
3,2.006,70.06573,Unknown,[M+H]+,similar chromatogram in higher mz_6446; simila...,0.059,False,NaN,NaN,NaN,...,170777,235339,64572,94949,38604,22061,1485,193892,1.407389e+05,152855.540782
4,5.374,70.06574,Unknown,[M+H]+,NaN,0.074,False,NaN,NaN,NaN,...,349644,97814,471873,417413,618698,46763,238769,636552,3.295410e+05,185665.566415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21382,1.548,894.59845,Unknown,[M+H]+,NaN,0.044,False,NaN,NaN,NaN,...,116451,0,0,116333,112553,151725,0,34850,4.458349e+04,64611.251535
21383,1.358,894.75348,Unknown,[M+Na]+,adduct linked to 21300_[M+H]+; adduct linked t...,0.882,True,NaN,NaN,NaN,...,955584,1475695,2477724,1242535,1975012,1133486,592283,1047666,1.034605e+06,733262.850754
21384,1.379,897.77325,Unknown,[M+H]+,NaN,0.088,False,NaN,NaN,NaN,...,820788,838276,1568833,1497694,1264850,1218647,834281,1014824,8.745012e+05,547126.069136


In [18]:
pos.sort_values?

In [34]:
pos[pos['Fill %'] > 0.3].sort_values(by=['Fill %'], ascending=False)

,Average Rt(min),Average Mz,Metabolite name,Adduct type,Post curation result,Fill %,MS/MS assigned,Reference RT,Reference m/z,Formula,...,20210831_KAO_HILIC_T1101M_20210413_10wk_RBG,20210831_KAO_HILIC_T1102M_20210327_8wk_FBG,20210831_KAO_HILIC_T1102M_20210402_9wk_FBG,20210831_KAO_HILIC_T1102M_20210406_9wk_RBG,20210831_KAO_HILIC_T1102M_20210410_10wk_FBG,20210831_KAO_HILIC_T1102M_20210413_10wk_RBG,20210831_KAO_HILIC_T112M_20210330_8wk_RBG,20210831_KAO_HILIC_T193M20210406_9wk_RBG,1,1.1
Alignment ID,,,,,,,,,,,,,,,,,,,,,
17314,1.468,500.37903,Unknown,[M+H]+,NaN,1.000,False,NaN,NaN,NaN,...,1474554,1364279,1213179,1439639,1542136,1414286,1226383,1467288,1.370382e+06,4.182820e+05
15913,1.479,442.33740,Unknown,[M+NH4]+,adduct linked to 15387_[M+H]+,1.000,False,NaN,NaN,NaN,...,720119,732460,934112,841123,924685,860294,1009061,746711,8.688247e+05,2.517021e+05
5190,2.040,195.12280,w/o MS2:Hydroxypyridin + C5H10NO or 6-Hydroxyp...,[M+H]+,adduct linked to 5998_[M+NH4]+; similar chroma...,1.000,True,1.80,195.11369,C10H14N2O2,...,11735219,7250998,9251689,8830203,10082091,11074877,7149584,8475507,7.708673e+06,3.698846e+06
10950,1.548,309.22705,Unknown,[M+H]+,adduct linked to 11915_[M+Na]+,1.000,True,NaN,NaN,NaN,...,1253270,1370001,1106375,1216346,1759354,1367105,1052589,1171096,1.198347e+06,2.834988e+05
12460,10.428,344.98465,Unknown,[M+Na]+,adduct linked to 11557_[M+H]+,1.000,False,NaN,NaN,NaN,...,3142368,560288,2089829,940517,1856257,448384,368447,544410,1.331541e+06,1.159642e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20528,16.988,750.89697,Unknown,[M+H]+,NaN,0.309,False,NaN,NaN,NaN,...,105467,85054,102911,101116,100714,120776,97923,152027,1.233550e+05,1.135095e+05
16828,2.904,476.27545,w/o MS2:LPE 18:3; PlaSMA ID-2582,[M+H]+,NaN,0.309,False,8.73,476.27560,C23H42NO7P,...,345372,116643,330724,335322,367000,543566,446877,407349,2.856763e+05,1.368371e+05
20580,6.941,758.56921,Unknown,[M+H]+,NaN,0.309,True,NaN,NaN,NaN,...,4598402,554621,2768398,1204539,1436548,1997302,1724818,854717,1.223739e+06,1.077343e+06


### rawfile names have some problems, so make a metabolomics file_grouping lookup table similar to lipids

In [90]:
bad_file = '20210831_KAO_HILIC_ACN'  # file size is only 800 kB and it failed to convert in MSConvert
typo_files = {'20210831_KAO_HILIC_T193M20210406_9wk_RBG': '20210831_KAO_HILIC_T1093M_20210406_9wk_RBG',
              '20210831_KAO_HILIC_T1092_20210410_10wk_FBG': '20210831_KAO_HILIC_T1092M_20210410_10wk_FBG',
              '20210831_KAO_HILIC_T112M_20210330_8wk_RBG': '20210831_KAO_HILIC_T1102M_20210330_8wk_RBG',
              '20210831_KAO_HILIC_T1092M_20210402_wk_FBG': '20210831_KAO_HILIC_T1092M_20210402_9wk_FBG'}
qc_files = list(metabolomics_seq[metabolomics_seq['File Name'].str.contains('human')]['File Name'].values)
blank_files = ['20210831_KAO_HILIC_ACN', '20210831_KAO_HILIC_ACN_20210831232610',
               '20210831_KAO_HILIC_extraction_blank_1', '20210831_KAO_HILIC_extraction_blank_1_20210901004002']

In [91]:
list(metabolomics_seq[metabolomics_seq['File Name'].str.contains('human')]['File Name'].values)

['20210831_KAO_HILIC_human_plasma_1',
 '20210831_KAO_HILIC_human_plasma_2',
 '20210831_KAO_HILIC_human_plasma_3',
 '20210831_KAO_HILIC_human_plasma_4',
 '20210831_KAO_HILIC_human_plasma_5']

In [92]:
[1,2] + [3,4]

[1, 2, 3, 4]

In [95]:
original_quant_filenames = list(metabolomics_seq[~metabolomics_seq['File Name'].isin(qc_files + blank_files)]['File Name'].values)
original_quant_filenames

modified_quant_filenames = []
for file in original_quant_filenames:
    if file in typo_files:
        modified_quant_filenames.append(typo_files[file])
    else:
        modified_quant_filenames.append(file)

file_sample_info = [(x, 
                     x.split('HILIC_T')[1].split('M_2021')[0], 
                     x.split('wk_')[0].split('_')[-1],
                     x.split('_')[-1]) for x in modified_quant_filenames]
df = pd.DataFrame(file_sample_info, columns=['modified_file_names', 'animal', 'week', 'bg_type'])
df['original_file_names'] = original_quant_filenames
df

,modified_file_names,animal,week,bg_type,original_file_names
0,20210831_KAO_HILIC_T1060M_20210401_9wk_RBG,1060,9,RBG,20210831_KAO_HILIC_T1060M_20210401_9wk_RBG
1,20210831_KAO_HILIC_T1076M_20210329_9wk_FBG,1076,9,FBG,20210831_KAO_HILIC_T1076M_20210329_9wk_FBG
2,20210831_KAO_HILIC_T1092M_20210413_10wk_RBG,1092,10,RBG,20210831_KAO_HILIC_T1092M_20210413_10wk_RBG
3,20210831_KAO_HILIC_T1101M_20210327_8wk_FBG,1101,8,FBG,20210831_KAO_HILIC_T1101M_20210327_8wk_FBG
4,20210831_KAO_HILIC_T1062M_20210405_10wk_FBG,1062,10,FBG,20210831_KAO_HILIC_T1062M_20210405_10wk_FBG
5,20210831_KAO_HILIC_T1101M_20210413_10wk_RBG,1101,10,RBG,20210831_KAO_HILIC_T1101M_20210413_10wk_RBG
6,20210831_KAO_HILIC_T1082M_20210408_10wk_RBG,1082,10,RBG,20210831_KAO_HILIC_T1082M_20210408_10wk_RBG
7,20210831_KAO_HILIC_T1092M_20210327_8wk_FBG,1092,8,FBG,20210831_KAO_HILIC_T1092M_20210327_8wk_FBG
8,20210831_KAO_HILIC_T1060M_20210322_8wk_FBG,1060,8,FBG,20210831_KAO_HILIC_T1060M_20210322_8wk_FBG
9,20210831_KAO_HILIC_T1093M_20210330_8wk_RBG,1093,8,RBG,20210831_KAO_HILIC_T1093M_20210330_8wk_RBG


In [94]:
df[df[0].str.contains('1092')]

,0,1,2,3,extra
2,20210831_KAO_HILIC_T1092M_20210413_10wk_RBG,1092,10,RBG,20210831_KAO_HILIC_T1092M_20210413_10wk_RBG
7,20210831_KAO_HILIC_T1092M_20210327_8wk_FBG,1092,8,FBG,20210831_KAO_HILIC_T1092M_20210327_8wk_FBG
13,20210831_KAO_HILIC_T1092M_20210406_9wk_RBG,1092,9,RBG,20210831_KAO_HILIC_T1092M_20210406_9wk_RBG
25,20210831_KAO_HILIC_T1092M_20210402_9wk_FBG,1092,9,FBG,20210831_KAO_HILIC_T1092M_20210402_wk_FBG
36,20210831_KAO_HILIC_T1092M_20210410_10wk_FBG,1092,10,FBG,20210831_KAO_HILIC_T1092_20210410_10wk_FBG
54,20210831_KAO_HILIC_T1092M_20210330_8wk_RBG,1092,8,RBG,20210831_KAO_HILIC_T1092M_20210330_8wk_RBG


In [69]:
list(metabolomics_seq['File Name'])

['20210831_KAO_HILIC_ACN',
 '20210831_KAO_HILIC_ACN_20210831232610',
 '20210831_KAO_HILIC_extraction_blank_1',
 '20210831_KAO_HILIC_extraction_blank_1_20210901004002',
 '20210831_KAO_HILIC_human_plasma_1',
 '20210831_KAO_HILIC_T1060M_20210401_9wk_RBG',
 '20210831_KAO_HILIC_T1076M_20210329_9wk_FBG',
 '20210831_KAO_HILIC_T1092M_20210413_10wk_RBG',
 '20210831_KAO_HILIC_T1101M_20210327_8wk_FBG',
 '20210831_KAO_HILIC_T1062M_20210405_10wk_FBG',
 '20210831_KAO_HILIC_T1101M_20210413_10wk_RBG',
 '20210831_KAO_HILIC_T1082M_20210408_10wk_RBG',
 '20210831_KAO_HILIC_T1092M_20210327_8wk_FBG',
 '20210831_KAO_HILIC_T1060M_20210322_8wk_FBG',
 '20210831_KAO_HILIC_T1093M_20210330_8wk_RBG',
 '20210831_KAO_HILIC_T1074M_20210401_9wk_RBG',
 '20210831_KAO_HILIC_T1102M_20210402_9wk_FBG',
 '20210831_KAO_HILIC_T1082M_20210329_9wk_FBG',
 '20210831_KAO_HILIC_T1092M_20210406_9wk_RBG',
 '20210831_KAO_HILIC_T1093M_20210410_10wk_FBG',
 '20210831_KAO_HILIC_T1060M_20210405_10wk_FBG',
 '20210831_KAO_HILIC_T1076M_20210405